# Descarga y conversión ERA5 → GrADS

Este notebook demuestra el flujo completo para:
1. Descargar datos meteorológicos ERA5 del CDS (Copernicus Data Store)
2. Convertir los archivos NetCDF a un formato optimizado para GrADS

**Requisitos previos**:
- Credenciales válidas para la API de CDS (archivo `.cdsapirc` configurado)
- Paquetes instalados: `era2grads`, `cdsapi`, `xarray`

In [1]:
# Importar las clases principales del paquete era2grads
# - PressureLevelDownloader: Para datos en niveles de presión atmosférica
# - SingleLevelDownloader: Para datos en superficie/nivel único
# - NCDFormatter: Para conversión y optimización de formatos
from era2grads import PressureLevelDownloader, NCDFormatter, SingleLevelDownloader
import os

### 1.1 Descargar datos niveles

Descarga de variables meteorológicas en niveles de presión específicos (500 hPa en este caso).

In [2]:
# Configurar y ejecutar la descarga para el componente zonal del viento (u) a 500 hPa
downloader = PressureLevelDownloader()
downloader.download(
    variables=['u_component_of_wind'],  # Variable meteorológica a descargar
    year=2023,                         # Año de interés
    month=6,                           # Mes (junio)
    days=[15],                         # Día específico (puede ser lista de días)
    pressure_levels=[500],             # Nivel de presión en hPa (500 hPa)
    area=[10, -80, -10, -60],         # Área geográfica [Norte, Oeste, Sur, Este]
    output_file="datos/viento_500hPa.nc"  # Ruta de salida para el archivo NetCDF
)

2025-04-17 15:48:36,138 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-17 15:48:36,139 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-04-17 15:48:36,821 INFO Request ID is 3fdb499c-8085-4f5d-8f65-c5ccba2eeb54
2025-04-17 15:48:37,295 INFO status has been updated to accepted
2025-04-17 15:48:51,906 INFO status has been updated to running
2025-04-17 15:48:59,769 INFO status has been updated to successful


### 1.2 Descargar datos superficie

Descarga de variables meteorológicas en superficie (viento a 10m en este caso).

In [3]:
# Configurar descarga para viento zonal a 10m sobre la misma región
downloader = SingleLevelDownloader()
downloader.download(
    variables=['10m_u_component_of_wind'],  # Viento zonal a 10m de altura
    year=2022,                             # Año diferente al ejemplo anterior
    month=6,                               # Mismo mes para consistencia
    days=[15],                             # Mismo día
    area=[10, -80, -10, -60],             # Misma área geográfica
    output_file="datos/viento_sup.nc"      # Archivo de salida diferente
)

2025-04-17 15:49:03,679 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-04-17 15:49:03,749 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-04-17 15:49:04,854 INFO Request ID is 3c1fd74e-aef3-47bc-8085-b5ef57604cdf
2025-04-17 15:49:05,104 INFO status has been updated to accepted
2025-04-17 15:49:39,402 INFO status has been updated to running
2025-04-17 15:49:56,769 INFO status has been updated to successful


### 2. Convertir a formato GrADS

Procesamiento de los archivos NetCDF para optimizarlos en GrADS:
- Renombrar variables (lat/lon) para compatibilidad
- Aplicar compresión eficiente
- Asegurar estructura de datos adecuada

In [4]:
# Procesar datos de niveles para GrADS
formatter = NCDFormatter()
formatter.grads_optimize(
    input_file="datos/viento_500hPa.nc",    # Archivo original descargado
    output_file="datos_grads/viento_grads.nc",  # Archivo optimizado
    rename_dict={'latitude': 'lat', 'longitude': 'lon'},  # Estándar GrADS
    cdo_compression="-f nc4c -z zip_4"      # Compresión balanceada
)

cdi  warning (cdfScanVarAttr): NetCDF: Variable not found - >isobaricInhPa<
cdi  warning (cdfScanVarAttr): NetCDF: Variable not found - >latitude<
cdi  warning (cdfScanVarAttr): NetCDF: Variable not found - >longitude<
cdi  warning (cdf_check_variables): Unsupported data type (char/string), skipped variable expver!
cdi  warning (cdfInqContents): Coordinates variable number can't be assigned!


'datos_grads/viento_grads.nc'

In [5]:
# Procesar datos de superficie para GrADS
formatter.grads_optimize(
    input_file="datos/viento_sup.nc",
    output_file="datos_grads/viento_sup_grads.nc",
    rename_dict={'latitude': 'lat', 'longitude': 'lon'},  # Mismo renombrado
    cdo_compression="-f nc4c -z zip_4"      # Misma compresión
)

cdi  warning (cdfScanVarAttr): NetCDF: Variable not found - >latitude<
cdi  warning (cdfScanVarAttr): NetCDF: Variable not found - >longitude<
cdi  warning (cdfInqContents): Coordinates variable number can't be assigned!
cdi  warning (cdfInqContents): Coordinates variable expver can't be assigned!


'datos_grads/viento_sup_grads.nc'